## Welcome to the Second Lab - Exercise: Advanced Agentic Design Patterns

This notebook extends the previous lab by adding the **Reflection Pattern** to improve response quality.

### Patterns used in the original lab:
1. **Multi-Model Comparison Pattern** - Comparing multiple models
2. **Judge/Evaluator Pattern** - Evaluation by a judge model

### New pattern added:
3. **Reflection Pattern** - Self-improvement of responses

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">New Pattern: Reflection</h2>
            <span style="color:#ff7800;">The Reflection Pattern allows a model to critique and improve its own response. This is particularly useful for complex tasks requiring nuance and precision.</span>
        </td>
    </tr>
</table>

In [1]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

# Always remember to do this!
load_dotenv(override=True)

True

In [2]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-1kYcH
Anthropic API Key exists and begins sk-ant-
Google API Key not set (and this is optional)
DeepSeek API Key not set (and this is optional)
Groq API Key not set (and this is optional)


## Step 1: Generate Initial Question (Multi-Model Pattern)

In [3]:
# Generate a challenging question for the models to answer

request = "Please come up with a challenging ethical dilemma that requires careful moral reasoning and consideration of multiple perspectives. "
request += "The dilemma should involve conflicting values and have no clear-cut answer. Answer only with the dilemma, no explanation."

messages = [{"role": "user", "content": request}]

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

question = response.choices[0].message.content
print("Generated Question:")
print(question)

Generated Question:
A wealthy philanthropist has developed a new drug that can cure a rare but fatal disease affecting a small population. However, the drug is expensive to produce and the philanthropist only has enough resources to manufacture a limited supply. At the same time, a competing pharmaceutical company has discovered the cure but plans to charge exorbitant prices, making it inaccessible for most patients. 

The philanthropist learns that if they invest their resources into manufacturing the drug, it can be distributed at a lower cost but only to a select few who are already on a waiting list, prioritizing those who are most likely to recover. Alternatively, the philanthropist could sell the formula to the competing company for a substantial profit, ensuring that a broader population can access the cure, albeit at high prices that many cannot afford.

The dilemma: Should the philanthropist prioritize the immediate health of a few individuals by providing the cure at a lower 

## Step 2: Get Initial Responses from Multiple Models

In [4]:
def get_initial_response(client, model_name, question, is_anthropic=False):
    """Get initial response from a model"""
    messages = [{"role": "user", "content": question}]
    
    if is_anthropic:
        response = client.messages.create(
            model=model_name, 
            messages=messages, 
            max_tokens=1000
        )
        return response.content[0].text
    else:
        response = client.chat.completions.create(
            model=model_name, 
            messages=messages
        )
        return response.choices[0].message.content

In [5]:
# Configure clients
openai_client = OpenAI()
claude_client = Anthropic() if anthropic_api_key else None
gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/") if google_api_key else None
deepseek_client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1") if deepseek_api_key else None
groq_client = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1") if groq_api_key else None

In [6]:
# Collect initial responses
initial_responses = {}
competitors = []

models = [
    ("gpt-4o-mini", openai_client, False),
    ("claude-3-7-sonnet-latest", claude_client, True),
    ("gemini-2.0-flash", gemini_client, False),
    ("deepseek-chat", deepseek_client, False),
    ("llama-3.3-70b-versatile", groq_client, False),
]

print("\n=== INITIAL RESPONSES ===\n")

for model_name, client, is_anthropic in models:
    if client:
        try:
            response = get_initial_response(client, model_name, question, is_anthropic)
            initial_responses[model_name] = response
            competitors.append(model_name)
            
            print(f"**{model_name}:**")
            display(Markdown(response))
            print("\n" + "="*50 + "\n")
        except Exception as e:
            print(f"Error with {model_name}: {e}")


=== INITIAL RESPONSES ===

**gpt-4o-mini:**


This ethical dilemma presents a challenging decision for the philanthropist, who must weigh the immediate health needs of a few individuals against the broader societal implications of drug distribution and access.

### Option 1: Prioritizing Immediate Health

If the philanthropist chooses to manufacture the drug and distribute it at a lower cost to those on the waiting list, they are directly addressing the pressing health needs of a select few individuals who are already vulnerable. This action prioritizes compassion and the moral obligation to help those who are suffering. By ensuring that the drug is available to those with the highest likelihood of recovery, the philanthropist demonstrates an ethical commitment to saving lives and reducing suffering in the short term.

However, this approach has limitations. By distributing the drug to only a small number of patients, the philanthropist may overlook other individuals who could benefit from the cure. Additionally, this solution does not address the systemic issue of access to healthcare and affordable medications for the larger population suffering from the disease.

### Option 2: Considering the Greater Good

On the other hand, selling the formula to the competing pharmaceutical company for a substantial profit could lead to a wider distribution of the drug, although at a higher price point that may make it inaccessible to many patients. In this scenario, the philanthropist uses their financial gain to potentially invest in other healthcare initiatives or research, thus contributing to the long-term improvement of medical care or addressing related health issues.

This choice raises ethical concerns regarding the prioritization of profit over compassion and the risk that many individuals will remain unable to afford the life-saving treatment. It also creates a tension between the ideals of philanthropy and the realities of the pharmaceutical industry, which often operates on profit motives rather than altruistic goals.

### Balancing the Two Options

A possible compromise could be for the philanthropist to negotiate a deal with the pharmaceutical company that ensures a tiered pricing structure, where those who can afford the drug pay more while discounts or alternative funding are provided for low-income patients. This could help bridge the gap between immediate health needs and wider access.

Ultimately, the decision comes down to the philanthropist's values and vision for their impact on public health. Do they prioritize saving a few lives in the short term or seek a more sustainable, albeit imperfect, solution that aims at broader access over a longer timeframe? The complexity of the dilemma emphasizes the need for thoughtful deliberation on how best to serve both individual health needs and the greater public good.



**claude-3-7-sonnet-latest:**


# The Philanthropist's Dilemma

This is a complex ethical dilemma that involves several important considerations:

## Key Ethical Tensions

- **Limited access at affordable prices** vs. **wider access at unaffordable prices**
- **Immediate relief for a few** vs. **potential long-term access for many**
- **Direct control over distribution** vs. **surrendering control to profit-motivated actors**

## Considerations for Manufacturing the Drug Directly

**Benefits:**
- Ensures the most vulnerable patients receive treatment based on medical need rather than ability to pay
- Maintains the philanthropist's ethical vision and control over distribution
- Sets a precedent for compassionate drug pricing

**Drawbacks:**
- Limited overall reach due to resource constraints
- Potentially slower scaling of production
- Many patients may receive no treatment at all

## Considerations for Selling to the Pharmaceutical Company

**Benefits:**
- Potentially greater production capacity and distribution reach
- The philanthropist could use profits to subsidize costs for those who cannot afford it
- Might accelerate further research and development

**Drawbacks:**
- Many patients would be excluded based on financial means
- Surrenders control over an essential medicine to profit-motivated decision-making
- Could establish a problematic precedent for pricing life-saving medications

This dilemma reflects broader tensions in healthcare ethics between utilitarian approaches (helping the most people) and justice-based approaches (ensuring fair access based on need rather than wealth).

There might be creative third options worth exploring, such as licensing agreements with price caps, creating a non-profit manufacturing entity, or partnering with governments to ensure broader affordable access.

## Step 3: NEW PATTERN - Reflection Pattern

In [7]:
def apply_reflection_pattern(client, model_name, original_question, initial_response, is_anthropic=False):
    """Apply the Reflection Pattern to improve a response"""
    
    reflection_prompt = f"""
You previously received this question:
{original_question}

Here was your initial response:
{initial_response}

Now, as a critical expert, analyze your own response:
1. What are the strengths of this response?
2. What important perspectives are missing?
3. Are there any biases or blind spots in the analysis?
4. How could you improve this response?

After this self-critique, provide an IMPROVED response that takes into account your observations.

Response format:
## Self-Critique
[Your critical analysis of the initial response]

## Improved Response
[Your revised and improved response]
"""
    
    messages = [{"role": "user", "content": reflection_prompt}]
    
    if is_anthropic:
        response = client.messages.create(
            model=model_name, 
            messages=messages, 
            max_tokens=1500
        )
        return response.content[0].text
    else:
        response = client.chat.completions.create(
            model=model_name, 
            messages=messages
        )
        return response.choices[0].message.content

In [8]:
# Apply Reflection Pattern
reflected_responses = {}

print("\n=== RESPONSES AFTER REFLECTION ===\n")

for model_name, client, is_anthropic in models:
    if client and model_name in initial_responses:
        try:
            reflected = apply_reflection_pattern(
                client, model_name, question, 
                initial_responses[model_name], is_anthropic
            )
            reflected_responses[model_name] = reflected
            
            print(f"**{model_name} - After Reflection:**")
            display(Markdown(reflected))
            print("\n" + "="*50 + "\n")
        except Exception as e:
            print(f"Error with reflection for {model_name}: {e}")


=== RESPONSES AFTER REFLECTION ===

**gpt-4o-mini - After Reflection:**


## Self-Critique
1. **Strengths of this Response:**
   - The response thoroughly outlines both options available to the philanthropist, providing a balanced view of the ethical implications of each choice.
   - It acknowledges the immediate health needs of affected individuals as well as the broader societal implications of drug distribution.
   - It introduces a potential compromise solution, which adds depth to the analysis and suggests a more nuanced approach to the dilemma.

2. **Important Perspectives Missing:**
   - The response does not adequately consider the potential operational and logistical challenges in manufacturing and distributing the drug at a lower cost, including regulatory hurdles and the scalability of production.
   - There is limited discussion on the emotional impact of the decision on the patients and their families, which could influence the philanthropist's considerations.
   - The perspective of other stakeholders, such as healthcare providers and ethicists, is not introduced.

3. **Biases or Blind Spots in the Analysis:**
   - The response may lean towards prioritizing compassion over economic pragmatism, possibly downplaying the complexities involved in pharmaceutical economics and the realities that arise from selling to a corporation with profit motives.
   - It assumes a binary choice rather than considering other stakeholder impacts and longer-term systemic solutions.

4. **How to Improve This Response:**
   - Include more contextual factors that might affect the decision, such as regulatory considerations, patient demographics, and healthcare infrastructure.
   - Expand on the emotional and psychological aspects of the decision-making process for both the philanthropist and the patients involved.
   - Address the potential for future societal implications if the competing company monopolizes the market after acquiring the formula.

## Improved Response
This ethical dilemma presents the philanthropist with a complex decision regarding how best to utilize limited resources to maximize the benefit for individuals suffering from a rare but fatal disease. The two primary options – providing a low-cost supply to a select few or selling the formula for broader but costly distribution – both highlight significant ethical considerations.

### Option 1: Prioritizing Immediate Health
By choosing to manufacture the drug at a lower cost for those on the waiting list, the philanthropist opts to directly address the urgent health needs of vulnerable individuals. This approach reflects a moral obligation to alleviate suffering and save lives in the short term. Prioritizing individuals with the highest likelihood of recovery can lead to tangible, immediate outcomes for those patients and their families.

However, there are operational challenges associated with this choice. Limited production capabilities may mean that only a fraction of those in need can actually receive the drug, leaving many others without hope. Additionally, this decision doesn't resolve the systemic issues within healthcare, such as overall treatment accessibility and drug pricing, which may persist if not tackled holistically.

### Option 2: Considering the Greater Good
Alternatively, selling the formula to the competing pharmaceutical company could result in wider distribution of the drug and potentially more patients benefiting from the cure, albeit at higher prices. This choice could finance further philanthropic efforts or investments in healthcare that might ultimately lead to broader long-term improvements in public health.

However, ethical concerns arise when considering the high pricing of the cure. The decision may disproportionately disadvantage lower-income patients, perpetuating healthcare inequities. Furthermore, there is the risk that this choice could enable the pharmaceutical company to monopolize treatment options, further exploitation in the industry.

### A Balanced Approach
To navigate this complex dilemma more thoughtfully, the philanthropist could explore a compromise by negotiating with the pharmaceutical company to establish a tiered pricing structure. This could create a system where the drug is offered at a reduced price for low-income patients, while ensuring sustainability for the company through higher prices for those who can afford them. Additionally, the philanthropist might advocate for a commitment from the company to invest in generics or alternative distribution methods to enhance accessibility.

### Conclusion
The choice ultimately hinges on the philanthropist's values and vision for their impact on public health. This decision requires careful consideration of immediate health benefits, long-term accessibility, and the emotional ramifications for affected individuals. By weighing the implications of each option and considering collaborative solutions, the philanthropist can work towards an outcome that promotes both individual care and broader societal well-being.



**claude-3-7-sonnet-latest - After Reflection:**


## Self-Critique

### Strengths of the initial response:
- Well-structured analysis that clearly outlines the ethical tensions
- Presents balanced considerations for both options
- Mentions potential third options beyond the binary choice
- Identifies the broader ethical frameworks at play (utilitarian vs. justice-based approaches)

### Missing perspectives:
1. **Stakeholder analysis**: The response lacks a thorough examination of all affected parties (patients, healthcare systems, future patients, etc.)
2. **Timeline considerations**: No discussion of short-term vs. long-term consequences beyond immediate access
3. **Public health impact**: Limited analysis of how each option affects overall public health outcomes
4. **Precedent-setting effects**: Inadequate exploration of how this decision might influence future pharmaceutical development and pricing
5. **Regulatory context**: No mention of potential government intervention, price controls, or other regulatory factors
6. **Global justice perspective**: No consideration of how this decision affects different regions/countries

### Biases and blind spots:
1. **False dichotomy**: Despite mentioning "third options," the analysis primarily treats this as a binary choice
2. **Western/developed-world bias**: Assumes a market-based healthcare system without considering different global contexts
3. **Individual-focused ethics**: Overemphasizes individual choice rather than institutional or systemic responsibilities
4. **Overly abstract**: The analysis lacks concrete examples or case studies that might inform the decision
5. **Neglect of power dynamics**: Doesn't address the power imbalance between corporations, individuals, and patients

### Improvement opportunities:
1. Provide a more nuanced spectrum of options beyond the binary choice
2. Include more stakeholder perspectives, particularly patient voices
3. Consider real-world case studies of similar pharmaceutical dilemmas
4. Address systemic issues in drug development and pharmaceutical pricing
5. Explore collaborative approaches that leverage multiple institutions
6. Discuss intellectual property rights and their ethical implications

## Improved Response

# The Philanthropist's Dilemma: A Multidimensional Ethical Analysis

This scenario presents not simply a binary choice but a complex ethical landscape involving multiple stakeholders, systemic factors, and competing values.

## Stakeholder Analysis

**Patients and families:**
- Those currently suffering need immediate access regardless of mechanism
- Future patients have interests in sustainable development of treatments
- Economic diversity among patients means affordability affects different groups unequally

**Healthcare systems:**
- Must allocate limited resources across competing priorities
- High-priced drugs can strain budgets and force difficult coverage decisions
- Precedents set now affect future negotiations with pharmaceutical companies

**Research community:**
- Incentives for developing treatments for rare diseases are influenced by such cases
- How intellectual property is handled affects future research priorities

## Ethical Frameworks Worth Considering

1. **Distributive justice**: Who should receive limited resources? What constitutes fair allocation?
2. **Rights-based approach**: Do patients have a right to life-saving medication regardless of cost?
3. **Consequentialist assessment**: Which option produces the best outcomes for the most people over time?
4. **Virtue ethics**: What would a virtuous philanthropist do in this situation?
5. **Global justice**: How does this decision affect healthcare equity across different regions?

## Spectrum of Options

Rather than two mutually exclusive choices, consider a spectrum of possibilities:

1. **Direct manufacturing with tiered pricing**: Manufacture independently but implement income-based pricing to maximize access while maintaining sustainability

2. **Conditional licensing**: License the formula with contractual price controls, distribution requirements, and accessibility guarantees

3. **Public-private partnership**: Collaborate with governments, NGOs, and selected pharmaceutical partners to ensure broad, affordable access

4. **Open-source approach**: Release the formula publicly with certain patent protections waived, while establishing a foundation to support manufacturing

5. **Hybrid distribution model**: Manufacture for highest-need populations while licensing to reach others, using licensing revenues to subsidize direct manufacturing

## Case Study Context

Similar dilemmas have occurred with treatments for HIV/AIDS, hepatitis C, and rare genetic disorders. The outcomes suggest:

- Maintaining some control over intellectual property while ensuring broad access often yields better public health outcomes than either extreme option
- Patient advocacy can significantly influence corporate behavior and pricing
- International differences in pricing and patent enforcement create complex dynamics
- Government intervention through negotiation, compulsory licensing, or regulation often becomes necessary

## Systems-Level Considerations

This dilemma exists within broader systemic issues:

- The current pharmaceutical development model creates inherent tensions between innovation, access, and affordability
- Rare disease treatments highlight market failures in drug development
- Healthcare financing systems vary globally, affecting how we should evaluate "accessibility"
- Intellectual property regimes may require reform to better balance innovation incentives with public health needs

## Recommended Approach

The philanthropist should pursue a hybrid strategy that:

1. Maintains sufficient control to ensure the most vulnerable patients receive treatment regardless of ability to pay

2. Leverages partnerships with multiple entities (pharmaceutical companies, governments, NGOs) to maximize production scale and geographic reach

3. Implements contractual safeguards on pricing, with particular attention to low and middle-income regions

4. Establishes a patient assistance foundation using a portion of any licensing revenues

5. Advocates for systemic reforms that would prevent such dilemmas in the future

This approach recognizes that the philanthropist's responsibility extends beyond the immediate distribution decision to include consideration of precedent-setting effects, stakeholder equity, and systemic change—balancing immediate needs with long-term public health impact.

## Step 4: Comparative Evaluation (Extended Judge Pattern)

In [9]:
def create_comparative_evaluation(question, initial_responses, reflected_responses):
    """Create a comparative evaluation of responses before/after reflection"""
    
    evaluation_prompt = f"""
You are evaluating the effectiveness of the "Reflection Pattern" on the following question:
{question}

For each model, you have:
1. An initial response
2. A response after self-reflection

Analyze and compare:
- Depth of analysis
- Consideration of multiple perspectives
- Nuance and sophistication of reasoning
- Improvement brought by reflection

MODELS TO EVALUATE:
"""
    
    for model_name in initial_responses:
        if model_name in reflected_responses:
            evaluation_prompt += f"""
## {model_name}

### Initial response:
{initial_responses[model_name][:500]}...

### Response after reflection:
{reflected_responses[model_name][:800]}...

"""
    
    evaluation_prompt += """
Respond with structured JSON:
{
    "general_analysis": "Your analysis of the Reflection Pattern's effectiveness",
    "initial_ranking": ["best initially ranked model", "second", "third"],
    "post_reflection_ranking": ["best ranked model after reflection", "second", "third"],
    "most_improved": "Which model improved the most",
    "insights": "Insights about the usefulness of the Reflection Pattern"
}
"""
    
    return evaluation_prompt

In [10]:
# Final evaluation
if initial_responses and reflected_responses:
    evaluation_prompt = create_comparative_evaluation(question, initial_responses, reflected_responses)
    
    judge_messages = [{"role": "user", "content": evaluation_prompt}]
    
    try:
        judge_response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=judge_messages,
        )
        
        evaluation_result = judge_response.choices[0].message.content
        print("\n=== FINAL EVALUATION ===\n")
        print(evaluation_result)
        
        # Try to parse JSON for structured display
        try:
            eval_json = json.loads(evaluation_result)
            print("\n=== STRUCTURED RESULTS ===\n")
            for key, value in eval_json.items():
                print(f"{key.replace('_', ' ').title()}: {value}")
        except:
            print("Could not parse JSON, raw output shown above")
            
    except Exception as e:
        print(f"Error during final evaluation: {e}")


=== FINAL EVALUATION ===

```json
{
    "general_analysis": "The Reflection Pattern effectively enhanced the depth of analysis and consideration of multiple perspectives in both models. However, the results differ in terms of sophistication and detail. The GPT-4 model provided initial observations that were relatively shallow but improved by incorporating logistical challenges and suggesting compromises during reflection. In contrast, Claude-3's initial response was more structured and sophisticated, covering a broader range of ethical frameworks, but still showed room for improvement regarding stakeholder analysis and long-term impacts.",
    "initial_ranking": ["claude-3-7-sonnet-latest", "gpt-4o-mini"],
    "post_reflection_ranking": ["claude-3-7-sonnet-latest", "gpt-4o-mini"],
    "most_improved": "gpt-4o-mini",
    "insights": "The Reflection Pattern revealed significant gaps in both models' initial analyses, encouraging deeper engagement with ethical implications and stakeholder

## Simple Before/After Comparison

In [11]:
# Display side-by-side comparison for each model
print("\n=== BEFORE vs AFTER COMPARISON ===\n")

for model_name in initial_responses:
    if model_name in reflected_responses:
        print(f"\n{'='*20} {model_name.upper()} {'='*20}\n")
        
        print("BEFORE REFLECTION:")
        print("-" * 50)
        print(initial_responses[model_name][:300] + "...")
        
        print("\nAFTER REFLECTION:")
        print("-" * 50)
        # Extract just the "Improved Response" section if it exists
        reflected = reflected_responses[model_name]
        if "## Improved Response" in reflected:
            improved_section = reflected.split("## Improved Response")[1].strip()
            print(improved_section[:400] + "...")
        else:
            print(reflected[:400] + "...")
        
        print("\n" + "="*70 + "\n")


=== BEFORE vs AFTER COMPARISON ===


==================== GPT-4O-MINI ====================

BEFORE REFLECTION:
--------------------------------------------------
This ethical dilemma presents a challenging decision for the philanthropist, who must weigh the immediate health needs of a few individuals against the broader societal implications of drug distribution and access.

### Option 1: Prioritizing Immediate Health

If the philanthropist chooses to manufa...

AFTER REFLECTION:
--------------------------------------------------
This ethical dilemma presents the philanthropist with a complex decision regarding how best to utilize limited resources to maximize the benefit for individuals suffering from a rare but fatal disease. The two primary options – providing a low-cost supply to a select few or selling the formula for broader but costly distribution – both highlight significant ethical considerations.

### Option 1: P...



==================== CLAUDE-3-7-SONNET-LATEST ==========

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Pattern Analysis</h2>
            <span style="color:#ff7800;">
            <b>Patterns used:</b><br/>
            1. <b>Multi-Model Comparison:</b> Comparing multiple models on the same task<br/>
            2. <b>Judge/Evaluator:</b> Using a model to evaluate performances<br/>
            3. <b>Reflection (NEW):</b> Self-critique and improvement of responses<br/><br/>
            <b>Possible experiments:</b><br/>
            - Iterate the Reflection Pattern multiple times<br/>
            - Add a "Debate Pattern" between models<br/>
            - Implement a "Consensus Pattern"
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial Applications</h2>
            <span style="color:#00bfff;">
            The <b>Reflection Pattern</b> is particularly valuable for:<br/>
            • Improving quality of complex analyses<br/>
            • Reducing bias in AI recommendations<br/>
            • Creating self-improving systems<br/>
            • Developing more robust AI for critical decisions<br/><br/>
            Use cases: Strategic consulting, risk analysis, ethical evaluation, medical diagnosis
            </span>
        </td>
    </tr>
</table>

## Additional Pattern Ideas for Future Implementation

In [12]:
# 1. Chain of Thought Pattern
"""
Add a pattern that asks models to show their reasoning step by step:

def apply_chain_of_thought_pattern(client, question):
    prompt = f\"
    Question: {question}
    
    Please think through this step by step:
    Step 1: [Identify the key issues]
    Step 2: [Consider different perspectives]
    Step 3: [Evaluate potential consequences]
    Step 4: [Provide reasoned conclusion]
    \"
    return get_response(client, prompt)
"""

# 2. Iterative Refinement Pattern
"""
Create a loop that progressively improves the response over multiple iterations:

def iterative_refinement(client, question, iterations=3):
    response = get_initial_response(client, question)
    for i in range(iterations):
        critique_prompt = f\"Improve this response: {response}\"
        response = get_response(client, critique_prompt)
    return response
"""

# 3. Debate Pattern
"""
Make two models debate their respective responses:

def create_debate(client1, client2, question):
    response1 = get_response(client1, question)
    response2 = get_response(client2, question)
    
    debate_prompt1 = f\"Argue against this position: {response2}\"
    debate_prompt2 = f\"Argue against this position: {response1}\"
    
    counter1 = get_response(client1, debate_prompt1)
    counter2 = get_response(client2, debate_prompt2)
    
    return counter1, counter2
"""

# 4. Consensus Building Pattern
"""
Attempt to create a consensus response based on all individual responses:

def build_consensus(all_responses, question):
    consensus_prompt = f\"
    Original question: {question}
    
    Here are multiple expert responses:
    {all_responses}
    
    Create a consensus response that incorporates the best insights from all responses
    while resolving contradictions.
    \"
    return get_response(openai_client, consensus_prompt)
"""

print("Exercise completed! Analyze the results to see the impact of the Reflection Pattern.")

Exercise completed! Analyze the results to see the impact of the Reflection Pattern.
